**Plan of the work:**

1)take model

2)take default CIFAR100 dataset

3)for fixed n_epochs measure acc after adv training 

4)construct dataset from CIFAR100 with following changes:

*   for hard classes - add their augmented images to the default dataset

5)for fixed n_epochs measure acc after adv training 

6)compare results from 3 and 5.

In [ ]:
from torchvision import datasets, models, transforms
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

import torch



data_transforms = transforms.Compose([
        transforms.RandomResizedCrop(256),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

In [ ]:
import random
import numpy as np

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

def clamp(X, lower_limit, upper_limit):
    return torch.max(torch.min(X, upper_limit), lower_limit)

In [ ]:
NUM_CLASSES = 100

from tqdm import tqdm 
import torch.nn.functional as F

def attack_pgd(model, X, y, epsilon, alpha, attack_iters, restarts, lower_limit, upper_limit, args=None):

    max_loss = torch.zeros(y.shape[0]).to(device)
    max_delta = torch.zeros_like(X).to(device)
    scaler = torch.cuda.amp.GradScaler()

    for _ in range(restarts):
        delta = torch.zeros_like(X).to(device)
        for i in range(len(epsilon)):
            delta[:, i, :, :].uniform_(-epsilon[i][0][0].item(), epsilon[i][0][0].item())
        delta.data = clamp(delta, lower_limit - X, upper_limit - X)
        delta.requires_grad = True

        for _ in range(attack_iters):
            with torch.cuda.amp.autocast():
                output = model(X + delta)
                index = torch.where(output.max(1)[1] == y)[0]
                if len(index) == 0:
                    break
                loss = F.cross_entropy(output, y)

            scaler.scale(loss).backward()
            grad = delta.grad.detach()

            d = delta[index, :, :, :]
            g = grad[index, :, :, :]
            d = clamp(d + alpha * torch.sign(g), -epsilon, epsilon)
            d = clamp(d, lower_limit - X[index, :, :, :], upper_limit - X[index, :, :, :])

            delta.data[index, :, :, :] = d
            delta.grad.zero_()

        all_loss = F.cross_entropy(model(X + delta), y, reduction='none').detach()
        max_delta[all_loss >= max_loss] = delta.detach()[all_loss >= max_loss]
        max_loss = torch.max(max_loss, all_loss)

    return max_delta


def evaluate_pgd(test_loader, model, attack_iters, restarts, epsilon, alpha, lower_limit, upper_limit, args=None):

    examples_per_class = {i : 0 for i in range(NUM_CLASSES)}
    correct_per_class = {i : 0 for i in range(NUM_CLASSES)}

    pgd_loss, pgd_acc = 0, 0
    n = 0
    model.eval()
    for i, (X, y) in enumerate(test_loader):
        X, y = X.to(device), y.to(device)
        pgd_delta = attack_pgd(model, X, y, epsilon, alpha, attack_iters, restarts, lower_limit, upper_limit, args=args)
        with torch.no_grad():
            output = model(X + pgd_delta)
            preds = output.max(1)[1]
            for i in range(NUM_CLASSES):
                examples_per_class[i] += len(y[y==i])
                correct_per_class[i] += (preds[y==i] == i).sum().item()
            loss = F.cross_entropy(output, y)
            pgd_loss += loss.item() * y.size(0)
            pgd_acc += (output.max(1)[1] == y).sum().item()
            n += y.size(0)
    acc_per_class = {}
    for i in range(NUM_CLASSES):
      if examples_per_class[i]:
        acc_per_class[i] = correct_per_class[i] / examples_per_class[i]
      else:
        acc_per_class[i] = -1
    return pgd_loss/n, pgd_acc/n,acc_per_class


def evaluate_standard(test_loader, model, args=None):
    
    examples_per_class = {i : 0 for i in range(NUM_CLASSES)}
    correct_per_class = {i : 0 for i in range(NUM_CLASSES)}
    
    test_loss, test_acc = 0, 0
    n = 0
    model.eval()
    with torch.no_grad():
        for i, (X, y) in tqdm(enumerate(test_loader), total=len(test_loader)):
            X, y = X.to(device), y.to(device)
            output = model(X)
            preds = output.max(1)[1]
            for i in range(NUM_CLASSES):
                examples_per_class[i] += len(y[y==i])
                correct_per_class[i] += (preds[y==i] == i).sum().item()
            loss = F.cross_entropy(output, y)
            test_loss += loss.item() * y.size(0)
            test_acc += (output.max(1)[1] == y).sum().item()
            n += y.size(0)
    acc_per_class = {}
    for i in range(NUM_CLASSES):
      if examples_per_class[i]:
        acc_per_class[i] = correct_per_class[i] / examples_per_class[i]
      else:
        acc_per_class[i] = -1
    return test_loss/n, test_acc/n,acc_per_class


In [ ]:
data_dir = 'cifar-100'

In [ ]:
train_transform = transforms.Compose([
        #transforms.RandomResizedCrop(32),
        transforms.RandomHorizontalFlip(),
        transforms.GaussianBlur(3,0.6),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

train_dataset1 = datasets.CIFAR100(data_dir, train=True, transform=train_transform, download=True)
train_dataset2 = datasets.CIFAR100(data_dir, train=True, transform=test_transform, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

a1 = np.array(train_dataset1[3][0]).transpose(1,2,0)
a2 = np.array(train_dataset2[3][0]).transpose(1,2,0)


fig,axes = plt.subplots(1,2,figsize=(10,5))
axes[0].imshow(a1)
axes[1].imshow(a2)

In [ ]:
import torch.nn as nn

class PreActBlock(nn.Module):
    '''Pre-activation version of the BasicBlock.'''
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super().__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)

        if (stride != 1) or (in_planes != self.expansion * planes):
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = F.relu(self.bn1(x))
        shortcut = self.shortcut(x) if hasattr(self, 'shortcut') else x
        out = self.conv1(out)
        out = self.conv2(F.relu(self.bn2(out)))
        out += shortcut
        return out


class PreActBottleneck(nn.Module):
    '''Pre-activation version of the original Bottleneck module.'''
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super().__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)

        if (stride != 1) or (in_planes != self.expansion * planes):
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = F.relu(self.bn1(x))
        shortcut = self.shortcut(out) if hasattr(self, 'shortcut') else x
        out = self.conv1(out)
        out = self.conv2(F.relu(self.bn2(out)))
        out = self.conv3(F.relu(self.bn3(out)))
        out += shortcut
        return out


class PreActResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super().__init__()
        self.in_planes = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.bn = nn.BatchNorm2d(512 * block.expansion)
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.relu(self.bn(out))
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def PreActResNet18(num_classes=10):
    return PreActResNet(PreActBlock, [2, 2, 2, 2], num_classes)

In [ ]:
hard_classes = [13,35,55,72,96,10,44,65,74,4]
easy_classes = [8,9,20,87,94,21,41,39,53,68]

hard_classes.index(55)

2

In [ ]:
X_all,y_all = train_dataset1.data, torch.LongTensor(train_dataset1.targets)
X_all_sampled, y_all_sampled = [], []

for c in torch.unique(y_all):
    X_all_c, y_all_c = X_all[y_all==c], y_all[y_all==c]
    
    if c in hard_classes:
        num_samples = len(X_all_c)
        sampled_indices = torch.randperm(num_samples)
        X_all_sampled.append(X_all_c[sampled_indices])
        y_all_sampled.append(torch.LongTensor([hard_classes.index(c)]*num_samples))
        
    
X_all_sampled = np.concatenate(X_all_sampled, axis=0)
y_all_sampled = list(torch.cat(y_all_sampled, dim=0).numpy())

X_all_sampled.shape

from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(X_all_sampled,y_all_sampled,test_size=0.25)

In [ ]:
def get_train_loader(data_dir, X, y, batch_size=32, augment=True, shuffle=True, args=None):

    train_transform = test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    train_dataset = datasets.CIFAR100(
            data_dir, train=True, transform=train_transform, download=True
        ) # we don't care what data here, we made it for getting appropriate transform
    train_dataset.data = X
    train_dataset.targets = y

    train_loader = torch.utils.data.DataLoader(
        dataset=train_dataset,
        batch_size=batch_size, shuffle=shuffle,
        pin_memory=True, num_workers=2,drop_last=True
    )
    return train_loader

train_loader = get_train_loader('cifar-train',x_train,y_train)
test_loader = get_train_loader('cifar-test',x_test,y_test)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
train_transform = test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
train_dataset = datasets.CIFAR100(
        'cifar_base_train', train=True, transform=train_transform, download=True
    )
test_dataset = datasets.CIFAR100(
        'cifar_base_test', train=False, transform=train_transform, download=True
    )

train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=32, shuffle=True,
    pin_memory=True, num_workers=2,drop_last=True
)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=32, shuffle=True,
    pin_memory=True, num_workers=2,drop_last=True
)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting cifar_base_train/cifar-100-python.tar.gz to cifar_base_train


  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting cifar_base_test/cifar-100-python.tar.gz to cifar_base_test


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'


weight_decay = 5e-4
epsilon = 8
batch_size = 32
alpha = 2
loss_weight = 0.3
fgsm_step = 1
delta_init = 'random'
model = PreActResNet18(num_classes=100).to(device)
model.train()

optimizer = torch.optim.Adam(model.parameters(),lr=3e-4,weight_decay=weight_decay)
scaler = torch.cuda.amp.GradScaler()
criterion = nn.CrossEntropyLoss()

data_mean = (0.5071, 0.4865, 0.4409)
data_std = (0.2673, 0.2564, 0.2762)

mu = torch.tensor(data_mean).view(3, 1, 1).to(device)
std = torch.tensor(data_std).view(3, 1, 1).to(device)
upper_limit = ((1 - mu) / std)
lower_limit = ((0 - mu) / std)
epsilon = (epsilon / 255.) / std
alpha = (alpha / 255.) / std

if delta_init == 'previous':
    delta = torch.zeros(batch_size, 3, 32, 32).to(device)

prev_robust_acc = 0.

In [ ]:
#model = PreActResNet18(num_classes=100).to(device)

optimizer = torch.optim.Adam(model.parameters(),lr=1e-4,weight_decay=weight_decay)

epochs = 7
loop = tqdm(range(epochs))
for epoch in loop:
    train_loss,train_acc,train_n = 0,0,0
    if epoch > 12:
      optimizer.param_groups[0]['lr'] = 0.5e-4

    model.train()
    for i, (X, y) in enumerate(train_loader):
        optimizer.zero_grad()
        X, y = X.to(device), y.to(device)
        if delta_init != 'previous':
            delta = torch.zeros_like(X).to(device)
        if delta_init == 'random':
            for j in range(len(epsilon)):
                delta[:, j, :, :].uniform_(-epsilon[j][0][0].item(), epsilon[j][0][0].item())
            delta.data = clamp(delta, lower_limit - X, upper_limit - X)
        if fgsm_step == 1:
            delta.requires_grad = True
            for _ in range(3):
                output = model(X + delta[:X.size(0)])
                loss = F.cross_entropy(output, y)
                scaler.scale(loss).backward()
                grad = delta.grad.detach()
                scaler.step(optimizer)
                scaler.update()
                delta.data = clamp(delta + alpha * torch.sign(grad), -epsilon, epsilon) 
                delta.data[:X.size(0)] = clamp(delta[:X.size(0)], lower_limit - X, upper_limit - X)
            delta = delta.detach()
            output = model(X + delta[:X.size(0)])
            loss = (1 - loss_weight) * criterion(output, y) + loss_weight * criterion(model(X), y)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
        train_loss += loss.item() * y.size(0)
        train_acc += (output.max(1)[1] == y).sum().item()
        train_n += y.size(0)
      
    model.eval()
    pgd_loss, pgd_acc, acc_cl_pgd_at = evaluate_pgd(test_loader, model, 1, 5, epsilon, alpha, 
                                  lower_limit, upper_limit)
    
    if pgd_acc >= 0.4:
      best_state_dict = model.state_dict()
      torch.save(best_state_dict,'model_default_good.pth')
      
    loop.set_description(f'train acc: {round(train_acc / train_n,3)} test acc: {pgd_acc}')

train acc: 0.336 test acc: 0.3880208333333333: 100%|██████████| 7/7 [39:28<00:00, 338.32s/it]


In [ ]:
pgd_loss, pgd_acc, acc_cl_pgd_at1 = evaluate_pgd(test_loader, model, 1, 5, epsilon, alpha, 
                                  lower_limit, upper_limit)
print(pgd_acc)
test_loss, test_acc, acc_cl_standart_at1 = evaluate_standard(test_loader, model)
print(test_acc)

0.3892227564102564


100%|██████████| 312/312 [00:12<00:00, 25.35it/s]

0.5045072115384616


In [ ]:
pgd_loss, pgd_acc, acc_cl_pgd_at1 = evaluate_pgd(test_loader, model, 1, 5, epsilon, alpha, 
                                  lower_limit, upper_limit)
print(pgd_acc)
test_loss, test_acc, acc_cl_standart_at1 = evaluate_standard(test_loader, model)
print(test_acc)

0.3837139423076923


100%|██████████| 312/312 [00:11<00:00, 26.72it/s]

0.49459134615384615


In [ ]:
best_state_dict = model.state_dict()
torch.save(best_state_dict,'model_comb_38.pth')


Training after 7 epochs on
base_data + extra hard-class data
 gives 27% accuracy (vs 33 on default dataset).

default + hard-class (augmented) - 0.384 (on attack test) and 0.495 (on default test)

default - 0.389 (on attack test) and 0.505 (on default test)

Addition of extra augmented images (with hard class labels) does not seem to improve performance of training.